# COMPILER USAGE NOTEBOOK
### Input file exemple
Please enter in between the quotes of variable string the wanted program
All this code will do is take that wanted string and save it in a given file named "newfile.txt"
Change the filename as you wish

In [2]:
string = """
#TIMESCALE
horizon: 3
step: 1

#NODE ABC
#PARAMETERS
c = 10 +10.01
v = (5)**c
#VARIABLES 
internal : a
output : b
#CONSTRAINTS
a[0]=1
a[t+1] = 3*a[0]
b[t*c+1] = 3*a*t
b[t]-(3*b[t+1] + (-a[t]))=-2*b[t]+1 
b-(3*b[t+1] + (-a))=-2*b+1 

#OBJECTIVE
min : a

#LINKS
A = B,C
A.pay = B.day"""
f = open("newfile.txt", "w")
f.write(string)
f.close()

### Lexer
The lexer takes as input a file seen as a stream of characters and converts into a stream of tokens
The tokens can be seen below with each token being the token name, Value, Line at which it was declared and the total number of characters read before the first character of this token


In [3]:
from lexer import tokenize_file

tokenize_file("newfile.txt")

LexToken(TIME,'#TIMESCALE',15,1)
LexToken(HORIZON,'horizon',16,12)
LexToken(COLON,':',16,19)
LexToken(INT,3,16,21)
LexToken(STEP,'step',17,23)
LexToken(COLON,':',17,27)
LexToken(INT,1,17,29)
LexToken(NODE,'#NODE',19,32)
LexToken(NAME,'ABC',19,38)
LexToken(PARAM,'#PARAMETERS',20,42)
LexToken(ID,'c',21,54)
LexToken(EQUAL,'=',21,56)
LexToken(INT,10,21,58)
LexToken(PLUS,'+',21,61)
LexToken(FLOAT,10.01,21,62)
LexToken(ID,'v',22,68)
LexToken(EQUAL,'=',22,70)
LexToken(LPAR,'(',22,72)
LexToken(INT,5,22,73)
LexToken(RPAR,')',22,74)
LexToken(POW,'**',22,75)
LexToken(ID,'c',22,77)
LexToken(VAR,'#VARIABLES',23,79)
LexToken(INTERNAL,'internal',24,91)
LexToken(COLON,':',24,100)
LexToken(ID,'a',24,102)
LexToken(OUTPUT,'output',25,104)
LexToken(COLON,':',25,111)
LexToken(ID,'b',25,113)
LexToken(CONS,'#CONSTRAINTS',26,115)
LexToken(ID,'a',27,128)
LexToken(LBRAC,'[',27,129)
LexToken(INT,0,27,130)
LexToken(RBRAC,']',27,131)
LexToken(EQUAL,'=',27,132)
LexToken(INT,1,27,133)
LexToken(ID,'a',28,135)
LexToke

### Parser
#### Classes
Takes as input the token stream and converts it to various a program class containing all the necessary information about the input file. 

The Program file is made up of : 
- A list of node objects
- A list of Links
- Information about time contained in timeclass

The Node object is made up of : 
- A name
- A list of Parameter objects
- A list of Variables 
- A list of Constraints 
- A list of Objective functions

A Parameter object contains:
- A name (parameter name)
- An expression (the rhs of the equality)

A Variable contains : 
- A name (Variable name)
- A parameter Type (Internal - Input - Output)

A constraint contains :
- A rhs expression 
- A sign (between "=","<=","<",">=",">")
- A lhs expression

An objective is made up of:
- A objective function between min or max
- An expression

An expression object contains : 
- A type between : 
    - 'literal' if it is a identifier, a float or an integer
    - '*' '/' '+' '-' if it is the according operation between two expression
    - 'u-' if it is a unary minus
- A list of child expressions
- A name containing the according identifier, float or integer if its type is literal

An expression object is build up as a tree of expression as follows, it starts from the bottom and goes upward:
- An integer, float or identifier is turned into a 'literal' type expression with the expression name containing the corresponding value
- If a unary minus is applied over an expression: 
    - A new expression 'u-' is created with only one child
- If a sum,division,soustraction,multiplication is applied between two expressions:
    - A new expression with the correspond type defined as the sign of the operation and two child expression
- If an expression is contained between parenthesis, simply the parenthesis are ignored

The priority of operations is given by a predefined precedence rule which corresponds to the usual mathematical precedence

#### Code
In the following code, the 'newfile.txt' file goes through the lexer and the corresponding stream is given to the parser which following a given grammar creates a full Program object. The full program object is printed. A more comprehensive way of seeing the different components of an object from the program class can be obtained by using the function 'to_string()'

In [8]:
from Myparser import parse_file


result = parse_file("newfile.txt")
print(result)
print("\n")
print(result.to_string())

[[ [ABCD , [ [c , [+ , [ 10 10.01]]] [v , [** , [ 5 c]]]] , [ [a , internal] [b , output]] , [ [= , a[0] , 1] [= , a[[+ , [ t 1]]] , [* , [ 3 a[0]]]] [= , b[[+ , [ [* , [ t c]] 1]]] , [* , [ 3 a]]] [= , [- , [ b[t] [+ , [ [* , [ 3 b[[+ , [ t 1]]]]] [u- , [ a[t]]]]]]] , [+ , [ [* , [ [u- , [ 2]] b[t]]] 1]]] [= , [- , [ b [+ , [ [* , [ 3 b[[+ , [ t 1]]]]] [u- , [ a]]]]]] , [+ , [ [* , [ [u- , [ 2]] b]] 1]]]] , []] [ABC , [ [c , [+ , [ 10 10.01]]] [v , [** , [ 5 c]]]] , [ [a , internal] [b , output]] , [ [= , a[0] , 1] [= , a[[+ , [ t 1]]] , [* , [ 3 a[0]]]] [= , b[[+ , [ [* , [ t c]] 1]]] , [* , [ 3 a]]] [= , [- , [ b[t] [+ , [ [* , [ 3 b[[+ , [ t 1]]]]] [u- , [ a[t]]]]]]] , [+ , [ [* , [ [u- , [ 2]] b[t]]] 1]]] [= , [- , [ b [+ , [ [* , [ 3 b[[+ , [ t 1]]]]] [u- , [ a]]]]]] , [+ , [ [* , [ [u- , [ 2]] b]] 1]]]] , [ [min,a]]]] , time: 3 step: 1 , [ [A , [ B C]] [A.pay , [ B.day]]]]


Full program
Time horizon : 3
Time step : 1
All the defined nodes : 
	Name : ABCD
		Parameters : [ [c , [

### Matrix  acquisition 

All the previous steps can be called by using main.py.
- The --lex option prints all the Tokens stream 
- The --parse option prints the syntax tree

main.py will also print the matrices for each constraint in each node. 
The formalism goes as follows: 
- Let us all the input variables of a given node by 
$$I = 
\begin{pmatrix}
u_1^t & u_2^t & ... & u_n^t
\end{pmatrix}
$$
- All the output variables are given by
$$
O = 
\begin{pmatrix}
o_1^t & o_2^t & ... & o_m^t
\end{pmatrix}
$$
- All the internal variables are given by
$$V = \begin{pmatrix}
v_1^t & v_2^t & ... & v_k^t
\end{pmatrix} $$

All the variables can concatenated into one single matrix $Y_{t\xrightarrow{}T}$ and evaluated for t=0,...,T-1 , where T denotes the considered time horizon
\begin{equation*}
    Y_{t\xrightarrow{}T} = \begin{pmatrix}
    I_{t=0} & V_{t=0} & O_{t=0}\\
    I_{t=1} & V_{t=1} & O_{t=1}\\
    \vdots & \vdots & \vdots\\
    I_{t=T-1} & V_{t=T-1} & O_{t=T-1}
    \end{pmatrix} = \begin{pmatrix}
    u_1^0 & ... & u_n^0 & v_1^0 & ... & v_k^0 & o_1^0 & ... & o_m^0 \\
    u_1^1 & ... & u_n^1 & v_1^1 & ... & v_k^1 & o_1^1 & ... & o_m^1 \\
    \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
    u_1^{T-1} & ... & u_n^{T-1} & v_1^{T-1} & ... & v_k^{T-1} & o_1^{T-1} & ... & o_m^{T-1}
    \end{pmatrix}
\end{equation*}

To simplify as in this matrix view being an input, output or an internal variable is not important we can rewrite $Y_{t\xrightarrow{}T}$ as

\begin{equation*}
    Y_{t\xrightarrow{}T} = \begin{pmatrix}
    x_1^0 & ... & x_{m+n+k}^0 \\
    x_1^1 & ... & x_{m+n+k}^1 \\
    \vdots & \vdots & \vdots\\
    x_1^{T-1} & ... & x_{m+n+k}^{T-1}
    \end{pmatrix}
\end{equation*}

The main goal will be to find the coefficient of a matrix A such that by applying a linear function, noted F, to multiplication of this matrix and the matrix $Y_{t\xrightarrow{}T}$ we get 
$$
F(A*Y_{t\xrightarrow{}T}) = c
$$
where c is the constant term in the considered constraint. 

A way to answer this problem is to consider a matrix A such that $i^{th}$ line is only considered (and therefore only multiplies) the corresponding $i^{th}$ column of matrix $Y_{t\xrightarrow{}T}$ in such a way that each constraint will be represented as a sum of the diagonal terms of the matrix $A*Y_{t\xrightarrow{}T}$. 

In otherwords, we get
$$
c = \begin{pmatrix}1 \\ \vdots \\ 1 \end{pmatrix}*\big( \sum_{i=1}^R \begin{pmatrix}\ddots & & \\ & 1 & \\ & & \ddots \end{pmatrix}*A*Y_{t\xrightarrow{}T}*\begin{pmatrix}\ddots & & \\ & 1 & \\ & & \ddots \end{pmatrix}\big) * \begin{pmatrix}1 & \dots & 1 \end{pmatrix}
$$
where R denotes the rank of the product $A*Y_{t\xrightarrow{}T}$ and the matrix $\begin{pmatrix}\ddots & & \\ & 1 & \\ & & \ddots \end{pmatrix}$ is sparse with only the $i^{th}$ term on the diagonal being 1. It is used to retrieve the $i^{th}$ number on the diagonal that is why a sum over all the diagonal numbers is needed.


In [2]:
!python main.py newfile.txt

a[0] b[0] 
a[1] b[1] 
a[2] b[2] 

CONSTRAINT : 0
t : 0
1 0 0 
0 0 0 
t : 1
1 0 0 
0 0 0 
t : 2
1 0 0 
0 0 0 
const : 1
CONSTRAINT : 1
t : 0
-3 1 0 
0 0 0 
t : 1
-3 0 1 
0 0 0 
t : 2
-3 0 0 
0 0 0 
const : 0
CONSTRAINT : 2
t : 0
0 0 0 
0 1 0 
t : 1
0 -3 0 
0 0 0 
t : 2
0 0 -6 
0 0 0 
const : 0
CONSTRAINT : 3
t : 0
1 0 0 
3 -3 0 
t : 1
0 1 0 
0 3 -3 
t : 2
0 0 1 
0 0 3 
const : 1
CONSTRAINT : 4
t : 0
1 0 0 
3 -3 0 
t : 1
0 1 0 
0 3 -3 
t : 2
0 0 1 
0 0 3 
const : 1
